<a href="https://colab.research.google.com/github/mkayanda/NLP_textgeneration/blob/main/chat_bot_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___
# Question and Answer Chat Bots

----

------

## Loading the Data

We will be working with the Babi Data Set from Facebook Research.

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698


In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("/content/drive/MyDrive/train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [ ]:
with open("/content/drive/MyDrive/test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

----

## Exploring the Format of the Data


In [ ]:
type(test_data)

list

In [ ]:
type(train_data)

list

In [ ]:
train_data_clean = pd.DataFrame(train_data)
test_data_clean = pd.DataFrame(test_data)

In [ ]:
train_data_clean.head(2)

,0,1,2
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no
1,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bathroom, ?]",no


In [ ]:
train_data_clean[2].unique()

array(['no', 'yes'], dtype=object)

In [ ]:
train_data_clean['0_clean'] = [' '.join(map(str, l)) for l in train_data_clean[0]]
train_data_clean['1_clean'] = [' '.join(map(str, l)) for l in train_data_clean[1]]
train_data_clean.head(2) 

,0,1,2,0_clean,1_clean
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no,Mary moved to the bathroom . Sandra journeyed ...,Is Sandra in the hallway ?
1,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bathroom, ?]",no,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bathroom ?


In [ ]:
test_data_clean['0_clean'] = [' '.join(map(str, l)) for l in test_data_clean[0]]
test_data_clean['1_clean'] = [' '.join(map(str, l)) for l in test_data_clean[1]]
test_data_clean.head(2) 

,0,1,2,0_clean,1_clean
0,"[Mary, got, the, milk, there, ., John, moved, ...","[Is, John, in, the, kitchen, ?]",no,Mary got the milk there . John moved to the be...,Is John in the kitchen ?
1,"[Mary, got, the, milk, there, ., John, moved, ...","[Is, John, in, the, kitchen, ?]",no,Mary got the milk there . John moved to the be...,Is John in the kitchen ?


In [ ]:
train_data_clean = train_data_clean.drop(labels=[0,1],axis=1)

In [ ]:
type(train_data_clean['0_clean'][0])

str

In [ ]:
test_data_clean = test_data_clean.drop(labels=[0,1],axis=1)

In [ ]:
train_data_clean[['0_clean', '1_clean']].values

array([['Mary moved to the bathroom . Sandra journeyed to the bedroom .',
        'Is Sandra in the hallway ?'],
       ['Mary moved to the bathroom . Sandra journeyed to the bedroom . Mary went back to the bedroom . Daniel went back to the hallway .',
        'Is Daniel in the bathroom ?'],
       ['Mary moved to the bathroom . Sandra journeyed to the bedroom . Mary went back to the bedroom . Daniel went back to the hallway . Sandra went to the kitchen . Daniel went back to the bathroom .',
        'Is Daniel in the office ?'],
       ...,
       ['Daniel went to the office . Sandra journeyed to the hallway . Daniel went back to the bedroom . Mary got the apple there . Sandra moved to the bathroom . Mary journeyed to the bedroom .',
        'Is Sandra in the hallway ?'],
       ['Daniel went to the office . Sandra journeyed to the hallway . Daniel went back to the bedroom . Mary got the apple there . Sandra moved to the bathroom . Mary journeyed to the bedroom . Mary put down the appl

In [ ]:
test_data_clean

,2,0_clean,1_clean
0,no,Mary got the milk there . John moved to the be...,Is John in the kitchen ?
1,no,Mary got the milk there . John moved to the be...,Is John in the kitchen ?
2,yes,Mary got the milk there . John moved to the be...,Is John in the garden ?
3,yes,Mary got the milk there . John moved to the be...,Is Daniel in the bathroom ?
4,no,Mary got the milk there . John moved to the be...,Is Daniel in the bedroom ?
...,...,...,...
995,no,Mary moved to the kitchen . Mary travelled to ...,Is Mary in the bedroom ?
996,no,Mary moved to the kitchen . Mary travelled to ...,Is Sandra in the kitchen ?
997,no,Mary moved to the kitchen . Mary travelled to ...,Is Mary in the bedroom ?
998,yes,Mary moved to the kitchen . Mary travelled to ...,Is Sandra in the garden ?


In [ ]:
train_data_clean

,2,0_clean,1_clean
0,no,Mary moved to the bathroom . Sandra journeyed ...,Is Sandra in the hallway ?
1,no,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bathroom ?
2,no,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the office ?
3,yes,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?
4,yes,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?
...,...,...,...
9995,no,Daniel went to the office . Sandra journeyed t...,Is Sandra in the kitchen ?
9996,yes,Daniel went to the office . Sandra journeyed t...,Is Daniel in the bedroom ?
9997,no,Daniel went to the office . Sandra journeyed t...,Is Sandra in the hallway ?
9998,no,Daniel went to the office . Sandra journeyed t...,Is Mary in the kitchen ?


In [ ]:
# .value changes the rows to arrays Ktrain expected the dataset in tuples. 
x_train = train_data_clean[['0_clean', '1_clean']].values
y_train = train_data_clean[2].values
x_test = test_data_clean[['0_clean', '1_clean']].values
y_test = test_data_clean[2].values

In [ ]:
x_train

array([['Mary moved to the bathroom . Sandra journeyed to the bedroom .',
        'Is Sandra in the hallway ?'],
       ['Mary moved to the bathroom . Sandra journeyed to the bedroom . Mary went back to the bedroom . Daniel went back to the hallway .',
        'Is Daniel in the bathroom ?'],
       ['Mary moved to the bathroom . Sandra journeyed to the bedroom . Mary went back to the bedroom . Daniel went back to the hallway . Sandra went to the kitchen . Daniel went back to the bathroom .',
        'Is Daniel in the office ?'],
       ...,
       ['Daniel went to the office . Sandra journeyed to the hallway . Daniel went back to the bedroom . Mary got the apple there . Sandra moved to the bathroom . Mary journeyed to the bedroom .',
        'Is Sandra in the hallway ?'],
       ['Daniel went to the office . Sandra journeyed to the hallway . Daniel went back to the bedroom . Mary got the apple there . Sandra moved to the bathroom . Mary journeyed to the bedroom . Mary put down the appl

In [ ]:
# IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train = list(map(tuple, x_train))
x_test = list(map(tuple, x_test))

In [ ]:
print(x_train[0])
print(y_train[0])

('Mary moved to the bathroom . Sandra journeyed to the bedroom .', 'Is Sandra in the hallway ?')
no


In [ ]:
!pip install ktrain

In [ ]:
import ktrain
from ktrain import text
# https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb
# https://huggingface.co/models
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=128, class_names=['no', 'yes'])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32) # lower bs if OOM occurs
learner.fit_onecycle(5e-5, 6)

preprocessing train...
language: en
sentence pairs detected


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/6
313/313 [==============================] - 136s 435ms/step - loss: 0.5053 - accuracy: 0.7258 - val_loss: 0.3461 - val_accuracy: 0.8460
Epoch 2/6
313/313 [==============================] - 136s 435ms/step - loss: 0.1597 - accuracy: 0.9391 - val_loss: 0.0190 - val_accuracy: 0.9950
Epoch 3/6
313/313 [==============================] - 136s 435ms/step - loss: 0.0456 - accuracy: 0.9847 - val_loss: 0.0032 - val_accuracy: 0.9980
Epoch 4/6
313/313 [==============================] - 136s 435ms/step - loss: 0.0117 - accuracy: 0.9976 - val_loss: 0.0042 - val_accuracy: 0.9990
Epoch 5/6
313/313 [==============================] - 136s 435ms/step - loss: 0.0093 - accuracy: 0.9981 - val_loss: 3.5385e-04 - val_accuracy: 1.0000
Epoch 6/6
313/313 [==============================] - 136s 435ms/step - loss: 0.0017 - accuracy: 0.9996 - val_loss: 2.5320e-04 - val_accuracy: 1.0000
